# Simulations

## Libraries

In [57]:
import numpy as np
import matplotlib.pyplot as plt
import uuid
from matplotlib.animation import FuncAnimation

## Resolution Configuration

In [58]:
RESOLUTION_4K_WIDTH = 3840
RESOLUTION_4K_HEIGHT = 2160
RESOLUTION_4K = (RESOLUTION_4K_WIDTH, RESOLUTION_4K_HEIGHT)

DOTS_PER_INCH = 200

FIGURE_DIMENSION = (RESOLUTION_4K_WIDTH / DOTS_PER_INCH, RESOLUTION_4K_HEIGHT / DOTS_PER_INCH)

FRAMES_PER_SECOND = 60.

TIME_PER_FRAME_IN_SECONDS = 1. / FRAMES_PER_SECOND
SIMULATION_TIME_BETWEEN_FRAMES_IN_MILLISECONDS = 1000. * TIME_PER_FRAME_IN_SECONDS

## Point Particle Properties

In [59]:
class Particle:
    def __init__(
        self,
        mass: float,
        charge: int,
        x_coordinate: float,
        y_coordinate: float,
        x_velocity: float,
        y_velocity: float):

        # Debugging:
        self.particle_id = uuid.uuid4()

        # Physics Properties:
        self.mass = mass
        self.charge = charge
        self.x_coordinate = x_coordinate
        self.y_coordinate = y_coordinate
        self.x_velocity = x_velocity
        self.y_velocity = y_velocity

        # Particle "History":
        self.x_coordinate_history = [self.x_coordinate,]
        self.y_coordinate_history = [self.y_coordinate,]
        self.x_velocity_history = [self.x_velocity,]
        self.y_velocity_history = [self.y_velocity,]

## Cartesian Coordinates

In [60]:
class CartesianCoordinates:
    def __init__(
        self,
        maximum_x_value: float,
        maximum_y_value: float,
        grid_spacing: float = 0.01):
        
        self.grid_spacing = grid_spacing
        self.minimum_x_value = 0.
        self.minimum_y_value = 0.
        self.maximum_x_value = maximum_x_value
        self.maximum_y_value = maximum_y_value

    def round_to_grid(self, number: float) -> float:
        """
        Rounds the given number to the nearest valid position on the grid.

        Args:
        - number (float): The number to round.

        Returns:
        - float: The rounded number.
        """
        return round(number / self.grid_spacing) * self.grid_spacing
    
    def generate_random_valid_starting_position(self) -> float:
        """
        Returns a random, valid starting position on the Cartesian
        coordinate grid.

        Returns:
        - (float, float): The (x, y) coordinates of the particle.
        """
        randomly_generated_x_coordinate = self.maximum_x_value * np.random.random()
        randomly_generated_x_coordinate_rounded = self.round_to_grid(randomly_generated_x_coordinate)

        randomly_generated_y_coordinate = self.maximum_y_value * np.random.random()
        randomly_generated_y_coordinate_rounded = self.round_to_grid(randomly_generated_y_coordinate)

        starting_coordinates = (randomly_generated_x_coordinate_rounded, randomly_generated_y_coordinate_rounded)
        return starting_coordinates 

## Simulation Settings

In [61]:
class Simulation:

    def __init__(
        self,
        total_timesteps: int,
        simulation_granularity: float,
        coordinate_grid: CartesianCoordinates,
        number_of_particles: int,
        verbose: bool = True):

        # Debugging:
        self.verbose = verbose

        # Simulation Settings:
        self.simulation_total_timesteps = total_timesteps
        self.simulation_step_size = simulation_granularity
        self.simulation_current_timestep = 0
        self.simulation_number_of_particles = number_of_particles
        self.simulation_coordinate_grid = coordinate_grid
        self.simulation_list_of_particles = []

        # Simulation Physics:
        self.physics_total_energy_history = []
        self.physics_kinetic_energy_history = []

    def generate_random_participating_particles(
        self,
        verbose: bool = True):

        for particle in range(self.simulation_number_of_particles):
            plus_minus_one = np.random.choice([-1., 1.], p = [0.5, 0.5])
            starting_xy_coordinates = self.simulation_coordinate_grid.generate_random_valid_starting_position()
            starting_xy_velocity = (plus_minus_one * np.random.random(), plus_minus_one * np.random.random())
            
            if verbose:
                print(f"> Particle will start at {starting_xy_coordinates} with velocity {starting_xy_velocity}")

            new_particle = Particle(
                mass = 1.0,
                charge = 0,
                x_coordinate = starting_xy_coordinates[0],
                y_coordinate = starting_xy_coordinates[1],
                x_velocity = starting_xy_velocity[0],
                y_velocity = starting_xy_velocity[1])
            
            self.simulation_list_of_particles.append(new_particle)
            
    def runge_kutta_4_time_evolve(
        self, 
        particle: Particle, 
        current_timestep: int):
        """
        """

        particle_mass = particle.mass
        particle_charge = particle.charge

        old_x_coordinate = particle.x_coordinate_history[current_timestep]
        old_y_coordinate = particle.y_coordinate_history[current_timestep]
        old_x_velocity = particle.x_velocity_history[current_timestep]
        old_y_velocity = particle.y_velocity_history[current_timestep]
        
        # COMPUTE NEW X-POSITION:
        x_position_k1 = old_x_velocity
        x_position_k2 = old_x_velocity + 0.5 * self.simulation_step_size * x_position_k1 
        x_position_k3 = old_x_velocity + 0.5 * self.simulation_step_size * x_position_k2
        x_position_k4 = old_x_velocity + self.simulation_step_size * x_position_k3
        new_x_coordinate = old_x_coordinate + self.simulation_step_size / 6. * (x_position_k1 + 2. * x_position_k2 + 2. * x_position_k3 + x_position_k4)

        # if new_x_coordinate >= self.simulation_coordinate_grid.maximum_x_value:
        #     new_x_coordinate = old_x_coordinate
        # if new_x_coordinate <= self.simulation_coordinate_grid.minimum_x_value:
        #     new_x_coordinate = old_x_coordinate

        # COMPUTE NEW Y-POSITION:
        y_position_k1 = old_y_velocity
        y_position_k2 = old_y_velocity + 0.5 * self.simulation_step_size * y_position_k1 
        y_position_k3 = old_y_velocity + 0.5 * self.simulation_step_size * y_position_k2
        y_position_k4 = old_y_velocity + self.simulation_step_size * y_position_k3
        new_y_coordinate = old_y_coordinate + self.simulation_step_size / 6. * (y_position_k1 + 2. * y_position_k2 + 2. * y_position_k3 + y_position_k4)

        # if new_y_coordinate >= self.simulation_coordinate_grid.maximum_y_value:
        #     new_y_coordinate = old_y_coordinate
        # if new_y_coordinate <= self.simulation_coordinate_grid.minimum_y_value:
        #     new_y_coordinate = old_y_coordinate

        # COMPUTE NEW X-VELOCITY:
        x_velocity_k1 = 0.
        x_velocity_k2 = 0. + 0.
        x_velocity_k3 = 0. + 0.
        x_velocity_k4 = 0. + 0.
        new_x_velocity = old_x_velocity + self.simulation_step_size / 6. * (x_velocity_k1 + 2. * x_velocity_k2 + 2. * x_velocity_k3 + x_velocity_k4)

        # COMPUTE NEW Y-VELOCITY:
        y_velocity_k1 = 0.
        y_velocity_k2 = 0. + 0.
        y_velocity_k3 = 0. + 0.
        y_velocity_k4 = 0. + 0.
        new_y_velocity = old_y_velocity + self.simulation_step_size / 6. * (y_velocity_k1 + 2. * y_velocity_k2 + 2. * y_velocity_k3 + y_velocity_k4)

        if self.verbose:
            print(f"> Evaluated new phase space point: {(new_x_coordinate, new_y_coordinate, new_x_velocity, new_y_velocity)}")

        return (new_x_coordinate, new_y_coordinate, new_x_velocity, new_y_velocity)

    def advance_timestep(self):
        """

        """

        for particle in self.simulation_list_of_particles:

            if self.verbose:
                print(f"> Now RK4 time-evolving Particle {particle.particle_id}")
            
            new_x, new_y, new_v_x, new_v_y = self.runge_kutta_4_time_evolve(particle, self.simulation_current_timestep)

            particle.x_coordinate_history.append(new_x)
            particle.y_coordinate_history.append(new_y)
            particle.x_velocity_history.append(new_v_x)
            particle.y_velocity_history.append(new_v_y)

    def compute_kinetic_energy(self):
        """
        """

        if self.verbose:
            print(f"> Now computing total Kinetic Energy.")

        total_kinetic_energy = 0.

        for particle in self.simulation_list_of_particles:
            particle_velocity_magnitude_squared = particle.x_velocity_history[self.simulation_current_timestep]**2 + particle.y_velocity_history[self.simulation_current_timestep]**2
            particle_kinetic_energy = 0.5 * particle.mass * particle_velocity_magnitude_squared
            total_kinetic_energy = total_kinetic_energy + particle_kinetic_energy
        
        self.physics_kinetic_energy_history.append(total_kinetic_energy)

    def compute_total_energy(self):

        if self.verbose:
            print(f"> Now computing Total Energy.")

        self.physics_total_energy_history.append(self.physics_kinetic_energy_history[self.simulation_current_timestep])

    
    def compute_physics(self):

        self.compute_kinetic_energy()
    
        self.compute_total_energy()


    def initialize_simulation(self):
        self.generate_random_participating_particles()

    
    def run_simulation(self):

        # (1): Initialize the simulation:
        if self.verbose:
            print(f"> Now initializing simulation...")
            
        self.initialize_simulation()

        for timestep in range(self.simulation_total_timesteps):

            if self.verbose:
                print(f"> Now running simulation for timestep #{timestep}.")

            self.advance_timestep()
            self.compute_physics()

            # At the end of the simulation timestep, increment the state:
            self.simulation_current_timestep = self.simulation_current_timestep + 1
                


# Setting Up Simulation

In [62]:
SIMULATION_NUMBER_OF_PARTICLES = 5
SIMULATION_TIMESTEP_GRANULARITY = 1
SIMULATION_TOTAL_TIMESTEPS = 100

SIMULATION_COORDINATE_SYSTEM = CartesianCoordinates(30., 30.)

simulation_instance = Simulation(
    SIMULATION_TOTAL_TIMESTEPS,
    SIMULATION_TIMESTEP_GRANULARITY,
    SIMULATION_COORDINATE_SYSTEM,
    SIMULATION_NUMBER_OF_PARTICLES,
    verbose = True)

simulation_instance.run_simulation()

> Now initializing simulation...
> Particle will start at (11.56, 0.37) with velocity (0.9471334013256106, 0.3427746009902829)
> Particle will start at (0.23, 28.830000000000002) with velocity (-0.32780626150176384, -0.7173407212730191)
> Particle will start at (8.620000000000001, 0.99) with velocity (0.7713311812651275, 0.6661863516376412)
> Particle will start at (9.75, 19.03) with velocity (-0.8754985960490927, -0.8271669375499493)
> Particle will start at (29.37, 24.36) with velocity (0.4602364952549254, 0.9444797976303182)
> Now running simulation for timestep #0.
> Now RK4 time-evolving Particle 7419da06-b9a8-440e-927a-85f939f25bdb
> Evaluated new phase space point: (13.17801956059792, 0.9555732766917332, 0.9471334013256106, 0.3427746009902829)
> Now RK4 time-evolving Particle 8554a42a-766e-44db-b37a-b9b6f2a5b52e
> Evaluated new phase space point: (-0.3300023633988465, 27.604542934491928, -0.32780626150176384, -0.7173407212730191)
> Now RK4 time-evolving Particle 225ed1bf-a622-41

In [65]:
HEXCOLOR_FIGURE_BACKGROUND = "#171717"
HEXCOLOR_SIMULATION_BACKGROUND = "#4a4a4a"
HEXCOLOR_DIAGNOSTIC_BACKGROUND = "#4a4a4a"

# (1): Initialize the Figure:
figure_object, (simulation_axes, diagnostic_axes) = plt.subplots(
    1, 
    2, 
    figsize = FIGURE_DIMENSION,
    dpi = DOTS_PER_INCH,
    gridspec_kw = {'width_ratios': [3, 1]},
    facecolor = HEXCOLOR_FIGURE_BACKGROUND)

# Always update rcParams:
plt.rcParams.update(plt.rcParamsDefault)

plt.rcParams["font.family"] = "serif"

plt.rcParams["mathtext.fontset"] = "cm" # https://matplotlib.org/stable/gallery/text_labels_and_annotations/mathtext_fontfamily_example.html

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 0.5
plt.rcParams['xtick.minor.size'] = 2.5
plt.rcParams['xtick.minor.width'] = 0.5
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['xtick.top'] = True    

# Set y axis
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['ytick.major.width'] = 0.5
plt.rcParams['ytick.minor.size'] = 2.5
plt.rcParams['ytick.minor.width'] = 0.5
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.right'] = True

def initialize_plot():

    simulation_axes.set_facecolor(HEXCOLOR_SIMULATION_BACKGROUND)
    diagnostic_axes.set_facecolor(HEXCOLOR_DIAGNOSTIC_BACKGROUND)

    simulation_axes.set_xlim(
        simulation_instance.simulation_coordinate_grid.minimum_x_value,
        simulation_instance.simulation_coordinate_grid.maximum_x_value)
    simulation_axes.set_ylim(
        simulation_instance.simulation_coordinate_grid.minimum_y_value,
        simulation_instance.simulation_coordinate_grid.maximum_y_value)

    # Add faint grid to simulation_axes
    simulation_axes.grid(color = 'black', linestyle = '--', linewidth = 0.5)

    # (): The aspect ratio must be equal otherwise the simulation appears sheard:
    simulation_axes.set_aspect('equal', adjustable = 'box')

    list_of_particles = []

    for particle in simulation_instance.simulation_list_of_particles:
        particle_starting_x_position = particle.x_coordinate_history[0]
        particle_starting_y_position = particle.y_coordinate_history[0]
        circle = plt.Circle(
            (particle_starting_x_position, particle_starting_y_position),
            radius = 1.0,
            color = "white",
            alpha = 0.8)
        simulation_axes.add_patch(circle)
        list_of_particles.append(circle)
    
    # Must return the elements (Artists) that will be updated in the animation
    return list_of_particles

circle_artists = initialize_plot()
print(circle_artists)

def modify_figure_data(frame):
    print(f"> Now rendering Frame #{frame}")

    for particle_index, circle in enumerate(circle_artists):
        new_center_x_coordinate = simulation_instance.simulation_list_of_particles[particle_index].x_coordinate_history[frame]
        new_center_y_coordinate = simulation_instance.simulation_list_of_particles[particle_index].y_coordinate_history[frame]
        circle.set_center((new_center_x_coordinate, new_center_y_coordinate))

    # List calculated "physics quantities" on the diagnostic_axes:
    diagnostic_axes.annotate(
        r"\sum_{i}^{N} T_{i} = {}",
        (1.,1.,),
        (1.,1.,)
    ) 

    # Must return the updated elements
    return False

# (2): Define the FuncAnimation
animation = FuncAnimation(
    fig = figure_object,
    func = modify_figure_data,
    frames = SIMULATION_TOTAL_TIMESTEPS,
    interval = SIMULATION_TIME_BETWEEN_FRAMES_IN_MILLISECONDS)

SIMULATION_NAME = "simulations"
animation.save(filename = f"{SIMULATION_NAME}_v5.gif", writer = "pillow")

[<matplotlib.patches.Circle object at 0x000001F28B47E090>, <matplotlib.patches.Circle object at 0x000001F2813D8200>, <matplotlib.patches.Circle object at 0x000001F281362600>, <matplotlib.patches.Circle object at 0x000001F2AF41DC40>, <matplotlib.patches.Circle object at 0x000001F2AF491D30>]
> Now rendering Frame #0
> Now rendering Frame #0
> Now rendering Frame #1
> Now rendering Frame #2
> Now rendering Frame #3
> Now rendering Frame #4
> Now rendering Frame #5
> Now rendering Frame #6
> Now rendering Frame #7
> Now rendering Frame #8
> Now rendering Frame #9
> Now rendering Frame #10
> Now rendering Frame #11
> Now rendering Frame #12
> Now rendering Frame #13
> Now rendering Frame #14
> Now rendering Frame #15
> Now rendering Frame #16
> Now rendering Frame #17
> Now rendering Frame #18
> Now rendering Frame #19
> Now rendering Frame #20
> Now rendering Frame #21
> Now rendering Frame #22
> Now rendering Frame #23
> Now rendering Frame #24
> Now rendering Frame #25
> Now rendering Fr

In [64]:
plt.plot(simulation_instance.simulation_list_of_particles[0].x_coordinate_history, simulation_instance.simulation_list_of_particles[0].y_coordinate_history)